# Create `sns.jointplots` for $\delta \phi_{P}$ and $A(\xi^{\prime}|\xi)$

In [ ]:
import os
import sys
import time
import pickle

import numpy as np
import pandas as pd
#import tensorflow as tf
import matplotlib.style as mplstyle
mplstyle.use('fast')
#import matplotlib as mpl
#mpl.use('fast')
#mpl.use('TkAgg')
import matplotlib.pyplot as plt
import seaborn as sns


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=3, linewidth=500, edgeitems=15, suppress=True)

import seaborn as sns
sns.set_palette('bright')


import matplotlib as mpl
import matplotlib.style as mplstyle

label_size = 9 
mpl.rcParams['xtick.labelsize'] = label_size 
mpl.rcParams['ytick.labelsize'] = label_size 


#from matplotlib import rc
#rc('text', usetex=False)

import utils.file_io as io

In [ ]:
from plotters.plot_observables import grid_plot, get_obs_dict
import utils.file_io as io

import pandas as pd
from plotters.plot_utils import load_pkl
from plotters.plot_observables import get_run_dirs, get_title_str
from lattice.lattice import u1_plaq_exact
import scipy
import datetime
#import matplotlib as mpl
#label_size = 9
#mpl.rcParams['xtick.labelsize'] = label_size 
#mpl.rcParams['ytick.labelsize'] = label_size 

In [ ]:
def set_size(width, fraction=1, subplot=[1, 1]):
    """ Set aesthetic figure dimensions to avoid scaling in latex.

    Parameters
    ----------
    width: float
            Width in pts
    fraction: float
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    #fig_height_in = fig_width_in * golden_ratio
    fig_height_in = fig_width_in * golden_ratio * (subplot[0] / subplot[1])

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim

In [ ]:
#sns.set_palette('bright', 100)
#colors = sns.color_palette()
#sns.set_style('white')
#sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
#sns.set_palette('bright')

In [ ]:
def therm_arr(arr, therm_frac=0.2):
    step_axis = np.argmax(arr.shape)
    num_steps = arr.shape[step_axis]
    therm_steps = int(therm_frac * num_steps)
    arr = np.delete(arr, np.s_[:therm_steps], axis=step_axis)
    
    return arr

def calc_tunneling_rate(charges):
    if charges.shape[0] > charges.shape[1]:
        charges = charges.T
    charges = np.around(charges)
    dq = np.abs(charges[:, 1:] - charges[:, :-1])
    tunneling_rate = np.mean(dq, axis=1)
    
    return dq, tunneling_rate

In [ ]:
import time
from plot_script import get_matching_log_dirs
t0 = time.time()
root_dir = os.path.abspath('../../gauge_logs/')
dates = [
    '2019_12_15',
    '2019_12_16',
    '2019_12_17',
    '2019_12_18',
    '2019_12_19',
    '2019_12_20',
    '2019_12_21',
    '2019_12_22',
    '2019_12_23',
    '2019_12_24',
    '2019_12_25',
    '2019_12_26',
    '2019_12_27',
    '2019_12_28',
    '2019_12_29',
    '2019_12_30',
    '2019_12_31',
    '2020_01_02',
    '2020_01_03',
    '2020_01_04',
    '2020_01_05',
    '2020_01_06',
    '2020_01_07',
    '2020_01_08',
    '2020_01_14',
    '2020_01_15',
    '2020_01_17',
    '2020_01_18'
]

log_dirs = []
for date in dates:
    ld = get_matching_log_dirs(date, root_dir=root_dir)
    for log_dir in ld:
        log_dirs += [log_dir]
    #log_dirs += [*ld]
    
print(len(log_dirs))
log_dirs

In [ ]:
from gauge_inference_np import inference_plots
for log_dir in log_dirs:
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    run_dirs = get_run_dirs(log_dir, filter_str='steps10000', runs_np=True)
    for run_dir in run_dirs:
        run_data_file = os.path.join(run_dir, 'run_data.pkl')
        energy_data_file = os.path.join(run_dir, 'energy_data.pkl')
        cond1 = os.path.isfile(run_data_file)
        cond2 = os.path.isfile(energy_data_file)
        if cond1 and cond2:
            #print(f'Making plots for {run_dir}...')
            run_params = load_pkl(os.path.join(run_dir, 'run_params.pkl'))
            run_data = load_pkl(run_data_file)
            energy_data = load_pkl(energy_data_file)
            dataset, energy_dataset = inference_plots(run_data, energy_data, params, run_params)

In [ ]:
from plotters.seaborn_plots import build_dataframes
#runs_np_arr = [True, True, True]
#filter_strs = ['steps5000', 'steps10000', 'steps20000']
runs_np_arr = [True]
filter_strs = ['steps10000']
dataframes_dict = {}
dataframes_bs_dict = {}
run_params_dict = {}
for runs_np, filter_str in zip(runs_np_arr, filter_strs):
    t0 = time.time()
    df_dict, df_bs_dict, rp_dict = build_dataframes(log_dirs,
                                                    filter_str=filter_str,
                                                    runs_np=runs_np)
    dataframes_dict[filter_str] = df_dict
    dataframes_bs_dict[filter_str] = df_bs_dict
    run_params_dict[filter_str] = rp_dict
    print(f'Time to complete: {time.time() - t0}s')

In [ ]:
dataframes_dict.keys()

In [ ]:
from plotters.seaborn_plots import gridplots

timestr = io.get_timestr()
runs_np = True
for filter_str in dataframes_dict.keys():
    dirstr = f'/home/foremans/gridplots/gridplots'
    if runs_np:
        dirstr += '_np'
    else:
        dirstr += '_tf'
    dirstr += f'_{filter_str}'
    dirstr += f"_{timestr['timestr']}"
    rootdir = os.path.abspath(dirstr)
    io.check_else_make_dir(rootdir)
    df_dict = dataframes_dict[filter_str]
    df_bs_dict = dataframes_bs_dict[filter_str]
    rp_dict = run_params_dict[filter_str]
    t0 = time.time()
    gridplots(log_dirs,
              df_dict=df_dict,
              df_bs_dict=df_bs_dict,
              rp_dict=rp_dict,
              rootdir=rootdir)

In [ ]:
from utils.file_io import rename_runs_np_dirs
rename_runs_np_dirs()

In [ ]:
from plotters.seaborn_plots import gridplots

timestr = io.get_timestr()
tstr = timestr['timestr']
dirstr = f'/home/foremans/gridplots_{filter_str}'
if runs_np:
    dirstr += '_np'
else:
    dirstr += '_tf'
timestr = io.get_timestr()
dirstr += f"_{timestr['timestr']}"
rootdir = os.path.abspath(dirstr)
io.check_else_make_dir(rootdir)

In [ ]:
gridplots(log_dirs,
          df_dict=df_dict,
          df_bs_dict=None,
          rp_dict=rp_dict,
          rootdir=rootdir)

In [ ]:
from os.path import join
import shutil
os.system("some_command with args")

base_dir = os.path.abspath('/home/foremans/DLHMC/l2hmc-qcd/gauge_logs/')
for root, dirs, files in os.walk(base_dir):
    fnames = [i.rstrip('.pdf') for i in files if i.endswith('.pdf')]
    in_files = [os.path.join(root, i) for i in files if i.endswith('.pdf')]
    if len(in_files) > 1:
        png_dir = os.path.join(root, 'pngs')
        io.check_else_make_dir(png_dir)
        out_files = [os.path.join(png_dir, f'{i}.png') for i in fnames]
        for inf, outf in zip(in_files, out_files):
            if not os.path.isfile(outf):
                print(f'in: {inf} --> out: {outf}\n')
                os.system(f'~/bin/pdftopng {inf} {outf}')

In [ ]:
from plotters.seaborn_plots import violinplots
fig, axes1, axes2 = violinplots(log_dirs,
                                df_dict=df_dict,
                                df_bs_dict=df_bs_dict,
                                rp_dict=rp_dict)

In [ ]:
%debug

In [ ]:
fig, axes = violinplots(log_dirs, df_dict=df_dict,
                        df_bs_dict=df_bs_dict, rp_dict=rp_dict)

In [ ]:
def get_train_weights(params):
    xsw = int(params['x_scale_weight'])
    xtw = int(params['x_translation_weight'])
    xqw = int(params['x_transformation_weight'])
    vsw = int(params['v_scale_weight'])
    vtw = int(params['v_translation_weight'])
    vqw = int(params['v_transformation_weight'])
    return (xsw, xtw, xqw, vsw, vtw, vqw)

def get_lf(log_dir):
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    lf = params['num_steps']
    return lf

In [ ]:
from plotters.plot_utils import bootstrap

#def bootstrap(data, reps=10000):
#    samples = []
#    percentiles = []
#    for x in data:
#        n = len(x)
#        xb = np.random.choice(x, (n, reps))
#        mb = xb.mean(axis=0)
#        mb.sort()
#        pp = np.percentile(mb, [2.5, 97.5])
#        samples.append(mb)
#        percentiles.append(pp)
#        
#    return np.array(samples), percentiles
def build_dataframes(run_dirs, data=None, data_bs=None, **kwargs):
    has_dx = False
    for run_dir in run_dirs:
        try:
            new_df, new_df_bs, run_params = get_observables(run_dir, **kwargs)
            if data is None:
                data = new_df
            else:
                data = pd.concat([data, new_df], axis=0).reset_index(drop=True)
                
            if data_bs is None:
                data_bs = new_df_bs
            else:
                data_bs = pd.concat([data_bs, new_df_bs], axis=0).reset_index(drop=True)
                
        except FileNotFoundError:
            continue
        
        if os.path.isfile(os.path.join(run_dir, 'observables', 'dx.pkl')):
            has_dx = True

    return data, data_bs, run_params, has_dx
        

def get_observables(run_dir, log_dir=None, n_boot=500,
                    therm_frac=0.25, nw_include=None, calc_stats=True):
    run_params = load_pkl(os.path.join(run_dir, 'run_params.pkl'))
    net_weights = tuple([int(i) for i in run_params['net_weights']])
    eps = run_params['eps']
    beta = run_params['beta']
    observables_dir = os.path.join(run_dir, 'observables')
    px = load_pkl(os.path.join(observables_dir, 'px.pkl'))
    px = np.squeeze(np.array(px))
    avg_px = np.mean(px)
    
    if nw_include is not None:
        keep_data = net_weights in nw_include
    else:
        keep_data = True
        
    if avg_px < 0.1 or not keep_data:
        print(f'INFO:Skipping! nw: {net_weights}, avg_px: {avg_px:.3g}')
        return None, None, run_params
    
    io.log(f'Loading data for net_weights: {net_weights}')
    
    def load_sqz(fname):
        data = load_pkl(os.path.join(observables_dir, fname))
        return np.squeeze(np.array(data))
    
    charges = load_sqz('charges.pkl')
    plaqs = load_sqz('plaqs.pkl')
    dplq = u1_plaq_exact(beta) - plaqs
    
    num_steps = px.shape[0]
    therm_steps = int(therm_frac * num_steps)
    steps = np.arange(therm_steps, num_steps)
    
    # NOTE: Since the number of tunneling events is computed as 
    # `dq = charges[1:] - charges[:-1]`,
    # we have that
    # `dq.shape[0] = num_steps - 1`.
    # Because of this, we drop the first step of `px` and `dplq` 
    # to enforce that they all have the same shape.
    px = px[therm_steps:]
    dplq = dplq[therm_steps:]
    charges = np.insert(charges, 0, 0, axis=0)
    charges = charges[therm_steps:]
    dq, _ = calc_tunneling_rate(charges)
    
    dq = dq.T
    
    dx_file = os.path.join(observables_dir, 'dx.pkl')
    if os.path.isfile(dx_file):
        print(f'Found dx_file: {dx_file}')
        dx = load_sqz('dx.pkl')
        dx = dx[therm_steps:]
        if dx.shape != px.shape:
            dx = None
    else:
        dx = None
        
    # dxf.shape = dxb.shape = (num_steps, batch_size, num_links)
    # Want to average over all links (last axis)
    dxf_file = os.path.join(observables_dir, 'dxf.pkl')
    if os.path.isfile(dxf_file):
        print(f'Found dxf_file: {dxf_file}')
        dxf = load_sqz('dxf.pkl')
        dxf = dxf.mean(axis=-1)
        dxf = dxf[therm_steps:]
        if dxf.shape != px.shape:
            dxf = None
    else:
        dxf = None
            
    dxb_file = os.path.join(observables_dir, 'dxf.pkl')
    if os.path.isfile(dxb_file):
        print(f'Found dxb_file: {dxf_file}')
        dxb = load_sqz('dxf.pkl')
        dxb = dxb.mean(axis=-1)
        dxb = dxf[therm_steps:]
        if dxb.shape != px.shape:
            dxb = None
    else:
        dxb = None
        
    
    if calc_stats:
        px_avg, px_err, px_ = bootstrap(px, n_boot=n_boot)
        dplq_avg, dplq_err, dplq_ = bootstrap(dplq, n_boot=n_boot)
        dq_avg, dq_err, dq_ = bootstrap(dq, n_boot=n_boot)
        px_ = px_.mean(axis=0)
        dplq_ = dplq_.mean(axis=0)
        dq_ = dq_.mean(axis=0)
        num_entries = len(dq_.flatten())
        data_bs = pd.DataFrame({
            'plaqs_diffs': dplq_.flatten(),
            'accept_prob': px_.flatten(),
            'tunneling_rate': dq_.flatten(),
            'net_weights': tuple([net_weights for _ in range(num_entries)]),
            'log_dir': np.array([log_dir for _ in range(num_entries)]),
        })
        if dx is not None:
            dx_avg, dx_err, dx_ = bootstrap(dx, n_boot=n_boot)
            dx_ = dx_.mean(axis=0)
            data_bs['dx'] = dx_.flatten()
        if dxf is not None:
            dxf_avg, dxf_err, dxf_ = bootstrap(dxf, n_boot=n_boot)
            dxf_ = dxf_.mean(axis=0)
            data_bs['dxf'] = dxf_.flatten()
        if dxb is not None:
            dxb_avg, dxb_err, dxb_ = bootstrap(dxb, n_boot=n_boot)
            dxb_ = dxb_.mean(axis=0)
            data_bs['dxb'] = dxb_.flatten()
            
    else:
        data_bs = None

    num_entries = len(dq.flatten())
    data = pd.DataFrame({
        'plaqs_diffs': dplq.flatten(),
        'accept_prob': px.flatten(),
        'tunneling_rate': dq.flatten(),
        'net_weights': tuple([net_weights for _ in range(num_entries)]),
        'log_dir': np.array([log_dir for _ in range(num_entries)]),
    })
    
    if dx is not None:
        data['dx'] = dx.flatten()
    if dxf is not None:
        data['dxf'] = dxf.flatten()
    if dxb is not None:
        data['dxb'] = dxb.flatten()
    
    return data, data_bs, run_params

In [ ]:
import time

t0 = time.time()
now = datetime.datetime.now()
day_str = now.strftime('%Y_%m_%d')
hour_str = now.strftime('%H%M')
time_str = f'{day_str}_{hour_str}'

m1 = ['x', 'v']
m2 = ['scale', 'translation', 'transformation']
matches = [f'{i}_{j}' for i in m1 for j in m2]

#colors = sns.set_palette('bright', 5)
colors = sns.color_palette('bright', 5)[::-1]
t0 = time.time()
dataframes_dict = {}
dataframes_bs_dict = {}
run_params_dict = {}
has_dx_dict = {}

log_dirs_sorted = sorted(log_dirs, key=get_lf, reverse=True)

In [ ]:
#for idx, log_dir in enumerate(sorted(log_dirs, key=get_lf, reverse=True)):
t0 = time.time()
for idx, log_dir in enumerate(log_dirs_sorted):
    if log_dir in dataframes_dict.keys():
        print(f'Already built dataframes for {log_dir}! Skipping!')
        continue
    if log_dir in dataframes_bs_dict.keys():
        print(f'Already built dataframes for {log_dir}! Skipping!')
        continue
    if log_dir in run_params_dict.keys():
        print(f'Already built dataframes for {log_dir}! Skipping!')
        continue
    else:
        print(f'Building dataframes for {log_dir}:\n')
        params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
        lf = params['num_steps']
        clip_value = params.get('clip_value', 0)
        #train_weights = tuple([
        #    int(params[key]) for key in params if any([m in key for m in matches])
        #])
        train_weights = get_train_weights(params)
        train_weights_str = ''.join((str(i) for i in train_weights))
        eps_fixed = params.get('eps_fixed', False)

        data = None
        data_bs = None
        n_boot = 1000
        therm_frac = 0.25

        try:
            run_dirs = sorted(get_run_dirs(log_dir))[::-1]
        except FileNotFoundError:
            continue

        nw_include = [
            (0, 0, 0, 0, 0, 0),
            (1, 0, 1, 1, 0, 1),
            (1, 0, 1, 1, 1, 1),
            (1, 1, 1, 1, 0, 1),
            (1, 1, 1, 1, 1, 1),
        ]
        #palette = dict(zip(nw_include, colors))

        data, data_bs, run_params, has_dx = build_dataframes(run_dirs,
                                                             data=data,
                                                             data_bs=data_bs,
                                                             n_boot=n_boot,
                                                             calc_stats=True,
                                                             therm_frac=therm_frac,
                                                             nw_include=nw_include)

        dataframes_dict[log_dir] = data
        dataframes_bs_dict[log_dir] = data_bs
        run_params_dict[log_dir] = run_params
        has_dx_dict[log_dir] = has_dx
print(f'Time to complete: {time.time() - t0}s') 

### Create plots:

For each `log_dir`, create a `pd.DataFrame` using all the inference data from `log_dir/runs/*`.

Loop over all `log_dirs` and create a `sns.PairGrid` to visualize:

- $\delta \phi_{P}$
- $A(\xi^{\prime}|\xi)$
- $\gamma$

In [ ]:
import matplotlib.ticker as ticker

def infer_cmap(color, palette='bright'):
    hues = sns.color_palette(palette)
    if color == hues[0]:
        return sns.light_palette(hues[0], 4, as_cmap=True)
    elif color == hues[1]:
        return sns.light_palette(hues[1], 4, as_cmap=True)
    elif color == hues[2]:
        return sns.light_palette(hues[2], 4, as_cmap=True)
    elif color == hues[3]:
        return sns.light_palette(hues[3], 4, as_cmap=True)
    elif color == hues[4]:
        return sns.light_palette(hues[4], 4, as_cmap=True)
    elif color == hues[5]:
        return sns.light_palette(hues[5], 4, as_cmap=True)

def kde_color_plot(x, y, **kwargs):
    palette = kwargs.pop('palette', 'bright')
    cmap = infer_cmap(kwargs['color'], palette=palette)
    ax = sns.kdeplot(x, y, cmap=cmap, **kwargs)
    ax.xaxis.set_major_locator(ticker.MaxNLocator(3))
    sns.despine(ax=ax, bottom=True, left=True)
    return ax

def kde_diag_plot(x, y, **kwargs):
    ax = sns.kdeplot(x, y, **kwargs)
    ax.xaxis.set_major_locator(ticker.MaxNLocator(3))
    return ax

def plot_pts(x, y, **kwargs):
    ax = plt.gca()
    ax.xaxis.set_major_locator(ticker.MaxNLocator(3))
    _ = ax.plot(x, y, **kwargs)
    #ax = sns.scatterplot(x, y, **kwargs)
    #sns.despine(ax=ax)
    return ax

#g = sns.PairGrid(df, hue='left', vars=['satisfaction_level', 'last_evaluation'], palette='Set1')
#g = g.map_upper(plt.scatter, s=1, alpha=0.5)
#g = g.map_lower(kde_color_plot)
#g = g.map_diag(sns.kdeplot, shade=True);

### Create grid plot using `sns.PairGrid`

In [ ]:
now = datetime.datetime.now()
day_str = now.strftime('%Y_%m_%d')
hour_str = now.strftime('%H%M')
time_str = f'{day_str}_{hour_str}'

m1 = ['x', 'v']
m2 = ['scale', 'translation', 'transformation']
matches = [f'{i}_{j}' for i in m1 for j in m2]

inv_colors = ['#007dff', '#fd971f',
              '#87ff00', '#f92672',
              '#909090', '#ffff00',
              '#70f0f0', '#A742EA']
inverse = sns.color_palette(inv_colors)
#sns.set_palette(inverse)
sns.set_palette('bright')

t0 = time.time()
#for idx, log_dir in enumerate(log_dirs):
keys = list(dataframes_dict.keys())[::-1]
#for idx, (log_dir, data) in enumerate(dataframes_dict.items()):
for idx, log_dir in enumerate(keys):
    io.log(f'log_dir: {log_dir}')
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    lf = params['num_steps']
    clip_value = params.get('clip_value', 0)
    
    train_weights = get_train_weights(params)
    train_weights_str = ''.join((str(i) for i in train_weights))
    eps_fixed = params.get('eps_fixed', False)
    
    data = dataframes_dict[log_dir]
    data_bs = dataframes_bs_dict[log_dir]
    
    #data = data.dropna()
    #data_bs = data_bs.dropna()
    
    run_params = run_params_dict[log_dir]
    #has_dx = has_dx_dict[log_dir]
    has_dx = hasattr(data, 'dx')

    n_boot = 1000
    therm_frac = 0.25

    eps = run_params['eps']
    with sns.axes_style('whitegrid'):
        mpl.rcParams['xtick.labelsize'] = 14 
        mpl.rcParams['ytick.labelsize'] = 14 
        vars_ = ['accept_prob', 'tunneling_rate', 'plaqs_diffs']
        #if has_dx:
        #    vars_ += ['dx']
        
        g = sns.PairGrid(data, hue='net_weights', vars=vars_,
                         palette='bright', diag_sharey=False, height=6)
        g = g.map_diag(sns.kdeplot, shade=True)
        #g = g.map_lower(plt.plot, markers=markers, ls='', #marker='x',
        #                rasterized=True, alpha=0.4)#, markersize=0.9)#, markeredgewidth=0.1, alpha=0.3)
        g = g.map_lower(sns.scatterplot, palette='bright')
        g = g.map_upper(kde_color_plot, palette='bright',
                        shade=False, gridsize=100, linewidths=1.5)
                        #shade=True, shade_lowest=False,
                        #alpha=0.75, gridsize=100) #, n_levels=8)#, linewidths=0.8)
        ##n_levels=30,
        g = g.add_legend(shadow=True)
        for ax in g.axes[:, 0]:
            ax.set_ylabel(ax.get_ylabel(), fontsize=16)
            
        for ax in g.axes[-1, :]:
            ax.set_xlabel(ax.get_xlabel(), fontsize=16)
        #legend = g.fig.legend(facecolor='#505050')
        #legend = plt.legend(facecolor='#505050')
        #legend = plt.legend()
        #plt.setp(legend.get_texts(), color='w')
        # Create title for plot
        title_str = (r"$N_{\mathrm{LF}} = $" + f'{lf}, '
                     r"$\varepsilon = $"  + f'{eps:.3g}')
        if eps_fixed:
            title_str += ' (fixed) '
        if any([tw == 0 for tw in train_weights]):
            tws = '(' + ', '.join((str(i) for i in train_weights_str)) + ')'
            title_str += (', ' + r"$\vec{\alpha}_{\mathrm{train}}=$" + f' {tws}')

        if params['clip_value'] > 0:
            title_str += f', clip: {clip_value}'
        g.fig.suptitle(title_str, y=1.03, fontsize=22)

    out_dir = os.path.abspath(f'/home/foremans/cooley_figures'
                              f'/combined_pairplots_{time_str}')
    io.check_else_make_dir(out_dir)

    fname = f'lf{lf}'
    if clip_value > 0:
        fname += f'_clip{int(clip_value)}'
    
    if eps_fixed:
        fname += f'_eps_fixed_'
    
    if any([tw == 0 for tw in train_weights]):
        #out_dir = os.path.join(out_dir, f'train_{train_weights_str}')
        fname += f'_train{train_weights_str}'
        
    id_str = log_dir.split('/')[-1].split('_')[-1]
    out_file = os.path.join(out_dir, f'{fname}_{id_str}.png')
    if os.path.isfile(out_file): 
        out_file = os.path.join(out_dir, f'{fname}_{id_str}_1.png')
        #out_file = os.path.join(out_dir, fname + '_1.png')
    io.log(f'INFO:Saving figure to: {out_file}')
    g.savefig(out_file, dpi=150, bbox_inches='tight')

    if not os.path.isfile(out_file):
        plt.savefig(out_file, dpi=150, bbox_inches='tight')
    print(f'Time spent plotting: {time.time() - t0:.3g}')
    io.log(80 * '-' + '\n')

### Create violin plots using `sns.violinplot`:

In [ ]:
colors = sns.color_palette(palette='muted', n_colors=len(nw_include))[::-1]
palette = dict(zip(nw_include, colors))

In [ ]:
now = datetime.datetime.now()
day_str = now.strftime('%Y_%m_%d')
hour_str = now.strftime('%H%M')
time_str = f'{day_str}_{hour_str}'
fontsize = 14
xticklabelsize = 14 
yticklabelsize = 14 
axeslabelsize = 18 
labelsize = 22
titlesize = 22

nw_include = [
    (0, 0, 0, 0, 0, 0),
    (1, 0, 1, 1, 0, 1),
    (1, 0, 1, 1, 1, 1),
    (1, 1, 1, 1, 0, 1),
    (1, 1, 1, 1, 1, 1),
]
#colors = sns.color_palette( 5)[::-1]
#palette = dict(zip(nw_include, colors))
colors = sns.color_palette(palette='muted', n_colors=len(nw_include))[::-1]
palette = dict(zip(nw_include, colors))
#sns.set_style("ticks")
#plt.style.use('seaborn-muted')

keys = list(dataframes_dict.keys())[::-1]
#for idx, (log_dir, data) in enumerate(dataframes_dict.items()):
for idx, log_dir in enumerate(keys):
    io.log(f'log_dir: {log_dir}')
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    lf = params['num_steps']
    clip_value = params.get('clip_value', 0)
    
    train_weights = get_train_weights(params)
    train_weights_str = ''.join((str(i) for i in train_weights))
    eps_fixed = params.get('eps_fixed', False)
    
    data = dataframes_dict[log_dir]
    data_bs = dataframes_bs_dict[log_dir]
    
    run_params = run_params_dict[log_dir]
    #has_dx = has_dx_dict[log_dir]
    has_dx = hasattr(data, 'dx')
    has_dxf = hasattr(data, 'dxf')
    has_dxb = hasattr(data, 'dxb')
    
    date_str = log_dir.split('/')[-2]
    y, m, d = date_str.split('_')
    y = int(y)
    m = int(m)
    d = int(d)

    old_dx = True
    if y == 2020 and m == 1 and d >= 4:
        old_dx = False
        
    #with sns.axes_style('darkgrid'):
    eps = run_params['eps']
    mpl.rcParams['xtick.labelsize'] = xticklabelsize 
    mpl.rcParams['ytick.labelsize'] = yticklabelsize 

    ncols = 3
    figsize = [20, 10]
    if has_dx:
        ncols += 1
        figsize[0] += 4
    if has_dxf:
        ncols += 1
        figsize[0] += 4
    if has_dxb:
        ncols += 1
        figsize[0] += 4

    figsize = tuple(figsize)

    fig, (axes0, axes1) = plt.subplots(nrows=2, ncols=ncols, figsize=figsize)
    # Initialise figure instance
    #fig, (axes0, axes1) = plt.subplots(nrows=2, ncols=ncols, sharey=True,
    #                                   figsize=set_size(width, subplot=[2, ncols]))

    axes = axes0.flatten()
    axes[0].axvline(x=0, ls=':', color='k')#, lw=0.9)
    axes[1].axvline(x=0, ls=':', color='k')#, lw=0.9)

    axes[0] = sns.violinplot(x='plaqs_diffs', y='net_weights', data=data, palette=palette, ax=axes[0])#, scale='width')#, saturation=1.)
    axes[1] = sns.violinplot(x='tunneling_rate', y='net_weights', data=data,  palette=palette, cut=0, ax=axes[1])#, scale='width')#, scale='width')#, saturation=1.)
    axes[2] = sns.violinplot(x='accept_prob', y='net_weights', data=data, palette=palette, ax=axes[2])#, scale='width')#, saturation=1.)
    if has_dx:
        axes[3] = sns.violinplot(x='dx', y='net_weights', data=data[data.dx < 10], palette=palette, ax=axes[3])#, scale='width')#, saturation=1.)
    if has_dxf:
        axes[4] = sns.violinplot(x='dxf', y='net_weights', data=data[data.dxf < 10], palette=palette, ax=axes[4])#, scale='width')#, saturation=1.)
    if has_dxb:
        axes[5] = sns.violinplot(x='dxb', y='net_weights', data=data[data.dxb < 10], palette=palette, ax=axes[5])#, scale='width')#, saturation=1.)


    for ax in axes:
        ax.set_ylabel('')
        ax.xaxis.set_major_locator(ticker.MaxNLocator(5))

    axes[0].set_xlabel(r"""$\delta \phi_{P}$""", fontsize=labelsize)
    axes[1].set_xlabel(r"""$\gamma$""", fontsize=labelsize)
    axes[2].set_xlabel(r"""$A(\xi^{\prime}|\xi)$""", fontsize=labelsize)

    l2_str = r"""$\|x^{(i+1)} - x^{(i)}\|^{2}_{2}$"""
    cos_str = r"""$1 - \cos(x^{(i+1)} - x^{(i)})$"""

    l2_strf = r"""$\|x_{f}^{(i+1)} - x_{f}^{(i)}\|^{2}_{2}$"""
    cos_strf = r"""$1 - \cos(x_{f}^{(i+1)} - x_{f}^{(i)})$"""

    l2_strb = r"""$\|x_{b}^{(i+1)} - x_{b}^{(i)}\|^{2}_{2}$"""
    cos_strb = r"""$1 - \cos(x_{b}^{(i+1)} - x_{b}^{(i)})$"""

    if has_dx:
        if old_dx:
            axes[3].set_xlabel(l2_str, fontsize=labelsize)

        else:
            axes[3].set_xlabel(cos_str, fontsize=labelsize)
    if has_dxf:
        if old_dx:
            axes[4].set_xlabel(l2_strf, fontsize=labelsize)
        else:
            axes[4].set_xlabel(cos_strf, fontsize=labelsize)
    if has_dxb:
        if old_dx:
            axes[5].set_xlabel(l2_strb, fontsize=labelsize)
        else:
            axes[5].set_xlabel(cos_strb, fontsize=labelsize)

    #axes[0].set_ylabel('')
    for ax in axes[1:]:
        ax.set_yticks([])
        ax.set_yticklabels([])

    axes = axes1.flatten()
    axes[0].axvline(x=0, ls=':', color='k')#, lw=0.9)
    axes[1].axvline(x=0, ls=':', color='k')#, lw=0.9)

    axes[0] = sns.violinplot(x='plaqs_diffs', y='net_weights', data=data_bs, palette=palette, ax=axes[0])#, scale='width')#, saturation=1.)
    axes[1] = sns.violinplot(x='tunneling_rate', y='net_weights', data=data_bs,  palette=palette, cut=0, ax=axes[1])#, scale='width')#, scale='width')#, saturation=1.)
    axes[2] = sns.violinplot(x='accept_prob', y='net_weights', data=data_bs, palette=palette, ax=axes[2])#, scale='width')#, saturation=1.)

    if has_dx:
        axes[3] = sns.violinplot(x='dx', y='net_weights', data=data_bs[data_bs.dx < 10], palette=palette, ax=axes[3])#, scale='width')
    if has_dxf:
        axes[4] = sns.violinplot(x='dxf', y='net_weights', data=data_bs[data_bs.dxf < 10], palette=palette, ax=axes[4])#, scale='width')#, saturation=1.)
    if has_dxb:
        axes[5] = sns.violinplot(x='dxb', y='net_weights', data=data_bs[data_bs.dxb < 10], palette=palette, ax=axes[5])#, scale='width')#, saturation=1.)


    for ax in axes:
        ax.set_ylabel('')
        ax.xaxis.set_major_locator(ticker.MaxNLocator(5))

    axes[0].set_xlabel(r"""$\langle\delta \phi_{P}\rangle$""", fontsize=labelsize)
    axes[1].set_xlabel(r"""$\langle\gamma\rangle$""", fontsize=labelsize)
    axes[2].set_xlabel(r"""$\langle A(\xi^{\prime}|\xi)\rangle$""", fontsize=labelsize)
    l2_str = r"""$\langle\|x^{(i+1)} - x^{(i)}\|^{2}_{2}\rangle$"""
    cos_str = r"""$\langle 1 - \cos(x^{(i+1)} - x^{(i)})\rangle$"""

    l2_strf = r"""$\langle\|x_{f}^{(i+1)} - x_{f}^{(i)}\|^{2}_{2}\rangle$"""
    cos_strf = r"""$\langle 1 - \cos(x_{f}^{(i+1)} - x_{f}^{(i)})\rangle$"""

    l2_strb = r"""$\langle \|x_{b}^{(i+1)} - x_{b}^{(i)}\|^{2}_{2}\rangle$"""
    cos_strb = r"""$\langle 1 - \cos(x_{b}^{(i+1)} - x_{b}^{(i)})\rangle$"""

    if has_dx:
        if old_dx:
            axes[3].set_xlabel(l2_str, fontsize=labelsize)

        else:
            axes[3].set_xlabel(cos_str, fontsize=labelsize)
    if has_dxf:
        if old_dx:
            axes[4].set_xlabel(l2_strf, fontsize=labelsize)
        else:
            axes[4].set_xlabel(cos_strf, fontsize=labelsize)
    if has_dxb:
        if old_dx:
            axes[5].set_xlabel(l2_strb, fontsize=labelsize)
        else:
            axes[5].set_xlabel(cos_strb, fontsize=labelsize)


    for ax in axes[1:]:
        ax.set_yticks([])
        ax.set_yticklabels([])

    fname = f'lf{lf}' #_{idx}'
    title_str = (r"$N_{\mathrm{LF}} = $" + f'{lf}, '
                 r"$\varepsilon = $"  + f'{eps:.3g}')

    eps_str = f'{eps:.3g}'.replace('.', '')
    fname += f'_e{eps_str}'

    if eps_fixed:
        title_str += ' (fixed) '
        fname += '_fixed'

    if any([tw == 0 for tw in train_weights]):
        tws = '(' + ', '.join((str(i) for i in train_weights_str)) + ')'
        title_str += (', ' + r"$\mathrm{nw}_{\mathrm{train}}=$" + f' {tws}')
        fname += f'_{train_weights_str}'


    if params['clip_value'] > 0:
        title_str += f', clip: {clip_value}'
        fname += f'_clip{clip_value}'
    fig.suptitle(title_str, fontsize=titlesize, y=1.025)
    out_dir = os.path.abspath(
        f'/home/foremans/cooley_figures/violinplots_{time_str}'
    )
    io.check_else_make_dir(out_dir)

    plt.tight_layout()
    #id_str = log_dir.split('/')[-1].split('_')[-1]
    #out_file = os.path.join(out_dir, f'{fname}.png')
    #id_str = f'{idx}'
    #out_file = os.path.join(out_dir, f'{fname}_{id_str}.png')
    out_file = os.path.join(out_dir, f'{fname}.pdf')
    if os.path.isfile(out_file): 
        id_str = f'{idx}'
        #out_file = os.path.join(out_dir, f'{fname}_{id_str}_{idx}.png')
        out_file = os.path.join(out_dir, f'{fname}_{id_str}.pdf')
    io.log(f'INFO:Saving figure to: {out_file}')
    plt.savefig(out_file, bbox_inches='tight')# dpi=200, bbox_inches='tight')

    if not os.path.isfile(out_file):
        plt.savefig(out_file, bbox_inches='tight')#, dpi=200, bbox_inches='tight')

In [ ]:
# Initialise figure instance
fig, ax = plt.subplots(2, 4, sharey=True, figsize=set_size(width, subplot=[2, 4]))

In [ ]:
palette = sns.color_palette('muted', 5)
with sns.axes_style('whitegrid'):
    mpl.rcParams['xtick.labelsize'] = 14
    mpl.rcParams['ytick.labelsize'] = 14 

    fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(15, 5))
    #ax1.axhline(y=0, ls=':', color='k', lw=0.8)
    axes = axes.flatten()[::-1]
    axes[1].axvline(x=0, ls=':', color='k')#, lw=0.9)
    axes[-1].axvline(x=0, ls=':', color='k')#, lw=0.9)
    axes[0] = sns.violinplot(x='accept_prob', y='net_weights', data=data, palette=palette, ax=axes[0])#, saturation=1.)
    axes[1] = sns.violinplot(x='tunneling_rate', y='net_weights', cut=0, data=data, palette=palette, ax=axes[1])#, saturation=1.)
    axes[-1] = sns.violinplot(x='plaqs_diffs', y='net_weights', data=data, palette=palette, ax=axes[-1])#, saturation=1.)

    for ax in axes:
        ax.set_ylabel('')
    ##for ax in axes:
    #    ax.set_ylabel(ax.get_ylabel(), fontsize=16)
    for ax in axes:
        ax.set_xlabel(ax.get_xlabel(), fontsize=16)
        
    fig.suptitle('testing', fontsize=22, y=1.)

    #axes[-1].set_xlabel(axes[-1].get_xlabel(), fontsize=16)

In [ ]:
now = datetime.datetime.now()
day_str = now.strftime('%Y_%m_%d')
hour_str = now.strftime('%H%M')
time_str = f'{day_str}_{hour_str}'

m1 = ['x', 'v']
m2 = ['scale', 'translation', 'transformation']
matches = [f'{i}_{j}' for i in m1 for j in m2]

#sns.set_palette('bright')
colors = sns.color_palette('bright', 5)[::-1]
t0 = time.time()
for idx, log_dir in enumerate(log_dirs):
    io.log(f'log_dir: {log_dir}')
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    lf = params['num_steps']
    clip_value = params.get('clip_value', 0)
    train_weights = tuple([
        int(params[key]) for key in params if any([m in key for m in matches])
    ])
    train_weights_str = ''.join((str(i) for i in train_weights))
    eps_fixed = params.get('eps_fixed', False)

    data = None
    n_boot = 1000
    therm_frac = 0.25

    try:
        run_dirs = sorted(get_run_dirs(log_dir))[::-1]
    except FileNotFoundError:
        continue
        
    nw_include = [
        (0, 0, 0, 0, 0, 0),
        (1, 0, 1, 1, 0, 1),
        (1, 0, 1, 1, 1, 1),
        (1, 1, 1, 1, 0, 1),
        (1, 1, 1, 1, 1, 1),
    ]
    palette = dict(zip(nw_include, colors))
    
    if os.path.isfile(os.path.join(run_dirs[0], 'observables', 'dxf.pkl')):
        data, run_params, has_dx = build_dataframes(run_dirs,
                                                    data=data,
                                                    n_boot=n_boot,
                                                    calc_stats=True,
                                                    therm_frac=therm_frac,
                                                    nw_include=nw_include)
        eps = run_params['eps']
        with sns.axes_style('darkgrid'):
            mpl.rcParams['xtick.labelsize'] = 10
            mpl.rcParams['ytick.labelsize'] = 10 
            if has_dx:
                ncols = data['net_weights'].nunique()
                fig, axes = plt.subplots(ncols=ncols, sharey=False)
                nws = data['net_weights'].unique()
                #colors = sns.color_palette('bright', ncols)
                for idx, (nw, ax) in enumerate(zip(nws, axes.flatten())):
                    #ax = sns.violinplot(y='dx', data=data[data.net_weights.isin([nw])], color=colors[idx], ax=ax, sharey=False, label=nw)
                    ax = sns.violinplot(y='dx', data=data[data.net_weights.isin([nw])], palette=palette, ax=ax, sharey=False, label=nw)
                for nw, ax in zip(nws, axes.flatten()):
                    ax.set_xlabel(nw, fontsize=12)
                for ax in axes[1:]:
                    ax.set_ylabel('')
                    
                axes[0].set_ylabel(axes[0].get_ylabel(), fontsize=14)
                #for idx, (nws, ax) in enumerate(zip(nws, axes.flatten())):
                #    ns.catplot(x="continent", y="lifeExp", hue="year", kind="point", 
                #    #data=df[df.continent.isin(['Asia','Europe'])]);
                #    ax = sns.violinplot(y='dx', data=data[data.net_weights.isin([nws])], palette='bright', ax=ax, hue='net_weights', sharey=False)
                #g = sns.catplot(y='dx', col='net_weights', data=data, kind='violin',
                #                hue='net_weights', dodge=False, palette='bright', sharey=False, )
                #for ax in g.axes[0]:
                #    ax.set_ylabel(ax.get_ylabel(), fontsize=18)
                #g = sns.FacetGrid(tips, hue="time", col="sex", height=4)
                #g.map(qqplot, "total_bill", "tip")
                #g.add_legend();
                #g = sns.FaceGrid()
                #>>> g = sns.catplot(x="time", y="pulse", hue="kind",
                #    col="diet", data=exercise)
                #fig, ax = plt.subplots()
                #ax = sns.violinplot(x='net_weights', y='dx', data=data, palette='bright', ax=ax)
                #sns.kdeplot()
                #ax.set_xlabel(ax.get_xlabel(), fontsize=12)

                title_str = (r"$N_{\mathrm{LF}} = $" + f'{lf}, '
                             r"$\varepsilon = $"  + f'{eps:.3g}')
                if eps_fixed:
                    title_str += ' (fixed) '
                if any([tw == 0 for tw in train_weights]):
                    tws = '(' + ', '.join((str(i) for i in train_weights_str)) + ')'
                    title_str += (', ' + r"$\mathrm{nw}_{\mathrm{train}}=$" + f' {tws}')

                if params['clip_value'] > 0:
                    title_str += f', clip: {clip_value}'
                #g.fig.suptitle(title_str, y=1.02, fontsize=16)
                fig.suptitle(title_str, y=1.03, fontsize=16)
                #plt.subplots_adjust(hspace=-0.1)
                out_dir = os.path.abspath(
                    f'/home/foremans/cooley_figures/dx_violinplots_{time_str}'
                )
                io.check_else_make_dir(out_dir)

                fname = f'lf{lf}'
                if clip_value > 0:
                    fname += f'_clip{int(clip_value)}'

                if eps_fixed:
                    fname += f'_eps_fixed_'

                if any([tw == 0 for tw in train_weights]):
                    fname += f'_train{train_weights_str}'

                plt.tight_layout()
                id_str = log_dir.split('/')[-1].split('_')[-1]
                #id_str = f'_{idx}'
                out_file = os.path.join(out_dir, f'{fname}_{id_str}.png')
                if os.path.isfile(out_file): 
                    out_file = os.path.join(out_dir, f'{fname}_{id_str}_1.png')
                    #out_file = os.path.join(out_dir, fname + '_1.png')
                io.log(f'INFO:Saving figure to: {out_file}')
                plt.savefig(out_file, dpi=150, bbox_inches='tight')

                if not os.path.isfile(out_file):
                    plt.savefig(out_file, dpi=150, bbox_inches='tight')

                print(f'Time spent plotting: {time.time() - t0:.3g}')
                io.log(80 * '-' + '\n')

    else:
        print('\n')
        print('INFO: `dx` not found. Skipping!')
        print('\n')

In [ ]:
ncols = data['net_weights'].nunique()
fig, axes = plt.subplots(ncols=ncols, sharey=False)
nws = data['net_weights'].unique()
colors = sns.color_palette('bright', ncols)
for idx, (nw, ax) in enumerate(zip(nws, axes.flatten())):
    ax = sns.violinplot(y='dx', data=data[data.net_weights.isin([nw])], color=colors[idx], ax=ax, sharey=False, label=nw)
    
    
for nw, ax in zip(nws, axes.flatten()):
    ax.set_xlabel(nw, fontsize=12)
for ax in axes[1:]:
    ax.set_ylabel('')
    
    
#plt.subplots_adjust(wspace=0.2)
plt.tight_layout()

In [ ]:
from config import COLORS
palette = {(1, 1, 1, 1, 1, 1): COLORS[0], (1, 1, 1, 1, 0, 1): COLORS[1], (1, 0, 1, 1, 1, 1): COLORS[0]}
g = sns.catplot(y='dx', col='net_weights', data=data, kind='violin',
                hue='net_weights', dodge=False, palette=palette, sharey=False)

In [ ]:
ax = sns.violinplot(y='dx', x='net_weights', data=data, palette='bright', )

In [ ]:
data['net_weights'].unique()

In [ ]:
s = pd.DataFrame(data['dx'], columns=data['net_weights'].unique())
s
#[(1, 1, 1, 1, 1, 1)]

In [ ]:
s

In [ ]:
for ax in g.axes[0]:
    ax.set_ylabel(ax.get_ylabel(), fontsize=18)
    

In [ ]:
g.axes[0][0]

In [ ]:
g.axes

In [ ]:
g.fig.suptitle

In [ ]:
with sns.axes_style('darkgrid'):
    g = sns.catplot(y='dx', col='net_weights', data=data, kind='violin', hue='net_weights', palette='bright', sharey=False)
    for ax in g.axes[0]:
        ax.set_ylabel(ax.get_ylabel(), fontsize=18)

In [ ]:
sns.kdeplot(data['dx'], hueshade=True)

In [ ]:
run_dir

In [ ]:
os.listdir(os.path.join(run_dir, 'observables'))

In [ ]:
with sns.axes_style('darkgrid'):
    mpl.rcParams['xtick.labelsize'] = 9
    mpl.rcParams['ytick.labelsize'] = 9

    fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(8, 10))
    ax1.axhline(y=0, ls=':', color='k', lw=0.8)
    ax2.axhline(y=0, ls=':', color='k', lw=0.8)
    ax3.axhline(y=0, ls=':', color='k', lw=0.8)
    ax1 = sns.violinplot(x='net_weights', y='tunneling_rate', data=data, palette='bright', ax=ax1)
    ax2 = sns.violinplot(x='net_weights', y='plaqs_diffs', data=data, palette='bright', ax=ax2)
    ax3 = sns.violinplot(x='net_weights', y='accept_prob', data=data, palette='bright', ax=ax3)
    ax1.set_xlabel('')
    ax2.set_xlabel('')
    #ax1.set_xticklabels([])
    #ax2.set_xticklabels([])

In [ ]:
ax.axh

In [ ]:
ax.set_yticklabels()

In [ ]:
p

In [ ]:
data

In [ ]:
>>> ax = sns.violinplot(x="day", y="total_bill", hue="smoker",
...                     data=tips, palette="muted")

In [ ]:
ax = sns.violinplot(x='net_weights', y='plaqs_diffs', data=data, palette='bright')

In [ ]:
sns.kdepl

In [ ]:
t0 = time.time()
g = sns.PairGrid(data, hue='net_weights', diag_sharey=False, palette='bright',
                 vars=['plaqs_diffs', 'accept_prob', 'tunneling_rate'], )
g = g.map_diag(sns.kdeplot, shade=True, gridsize=5)
g = g.map_lower(plt.plot, ls='', marker='+', rasterized=True, markeredgewidth=0.1)#, alpha=1.)
g = g.map_upper(sns.kdeplot, shade=False, gridsize=5, linewidths=0.7)
print(f'{time.time() - t0:.3g}s')

In [ ]:
g = sns.PairGrid(data, hue='net_weights', diag_sharey=False, palette='husl')
g = g.map_upper(plt.plot, ls='', marker=',', rasterized=True)#, alpha=1.)
try:
    g = g.map_diag(sns.kdeplot, shade=True, gridsize=100)
    g.add_legend()
    g = g.map_lower(sns.kdeplot, shade=False, gridsize=50)
except:
    g = g.map_diag(plt.hist, histtype='step', alpha=0.6, density=True)
    g.add_legend()

In [ ]:
for log_dir in log_dirs:
    io.log(f'log_dir: {log_dir}')
    run_dirs = sorted(get_run_dirs(log_dir))
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    lf = params['num_steps']
    clip_value = params.get('clip_value', 0)
    train_weights = tuple([
        int(params[key]) for key in params if any([m in key for m in matches])
    ])
    train_weights_str = ''.join((str(i) for i in train_weights))
    cmap = sns.light_palette(colors[idx], as_cmap=True)
    for run_dir in run_dirs:
        t1 = time.time()
        data, run_params = get_observables(run_dir, n_boot=1000, therm_frac=0.2)
        key = tuple([int(i) for i in run_params['net_weights']])
        eps = run_params['eps']
        if data is None:
            continue

        g = sns.PairGrid(data, hue='net_weights', diag_sharey=False, palette='husl')
        g = g.map_upper(plt.plot, ls='', marker=',', rasterized=True)#, alpha=1.)
        try:
            g = g.map_diag(sns.kdeplot, shade=True, gridsize=100)
            g.add_legend()
            g = g.map_lower(sns.kdeplot, shade=False, gridsize=50)
        except:
            g = g.map_diag(plt.hist, histtype='step', alpha=0.6, density=True)
            g.add_legend()

        # Create title for plot
        title_str = (r"$N_{\mathrm{LF}} = $" + f'{lf}, '
                     r"$\varepsilon = $"  + f'{eps:.3g}')
        if params['clip_value'] > 0:
            title_str += f', clip: {clip_value}'
        title_str += f', nw: {key}'
        g.fig.suptitle(title_str, y=1.02, fontsize='x-large')

In [ ]:
import time
from plotters.plot_utils import bootstrap
import shutil
import datetime
from plotters.plot_utils import load_pkl
from plotters.plot_observables import get_run_dirs, get_title_str
from lattice.lattice import u1_plaq_exact
import matplotlib.patches as patches
np.set_printoptions(edgeitems=3,infstr='inf',
                    linewidth=75, nanstr='nan', precision=8,
                    suppress=False, threshold=1000, formatter=None)

now = datetime.datetime.now()
day_str = now.strftime('%Y_%m_%d')
hour_str = now.strftime('%H%M')
time_str = f'{day_str}_{hour_str}'
from config import COLORS

colors = 100 * [*COLORS]

m1 = ['x', 'v']
m2 = ['scale', 'translation', 'transformation']
matches = [f'{i}_{j}' for i in m1 for j in m2]

t0 = time.time()
for idx, log_dir in enumerate(log_dirs):
    io.log(f'log_dir: {log_dir}')
    run_dirs = sorted(get_run_dirs(log_dir))
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    lf = params['num_steps']
    clip_value = params.get('clip_value', 0)
    train_weights = tuple([
        int(params[key]) for key in params if any([m in key for m in matches])
    ])
    train_weights_str = ''.join((str(i) for i in train_weights))
    cmap = sns.dark_palette(colors[idx], as_cmap=True)
    for run_dir in run_dirs:
        t1 = time.time()
        data, run_params = get_observables(run_dir, n_boot=1000, therm_frac=0.2)
        key = tuple([int(i) for i in run_params['net_weights']])
        eps = run_params['eps']
        if data is None:
            continue
        
        g = sns.PairGrid(data, diag_sharey=False)
        g = g.map_upper(plt.plot, color=colors[idx], ls='', marker=',', rasterized=True)#, alpha=1.)
        try:
            g = g.map_diag(sns.kdeplot, shade=True, color=colors[idx], gridsize=100)
            g = g.map_lower(sns.kdeplot, shade=False, cmap=cmap, Nchunk=5, gridsize=50)
        except:
            g = g.map_lower(plt.hist, histtype='step', color=colors[idx], alpha=0.6, ec=colors[idx], density=True)
            g = g.map_diag(plt.hist, histtype='step', color=colors[idx], alpha=0.6, ec=colors[idx], density=True,)
            
        # Create title for plot
        title_str = (r"$N_{\mathrm{LF}} = $" + f'{lf}, '
                     r"$\varepsilon = $"  + f'{eps:.3g}')
        if params['clip_value'] > 0:
            title_str += f', clip: {clip_value}'
        title_str += f', nw: {key}'
        g.fig.suptitle(title_str, y=1.02, fontsize='x-large')
        
        out_dir = os.path.abspath(f'/home/foremans/cooley_figures/pairplots_{time_str}')
        if any([tw == 0 for tw in train_weights]):
            out_dir = os.path.join(out_dir, f'train_{train_weights_str}')
        io.check_else_make_dir(out_dir)
        
        nw_str = ''.join((str(int(i)) for i in key))
        fname = f'lf{lf}_'
        if clip_value > 0:
            fname += f'clip{int(clip_value)}_'
        fname += f'_{nw_str}'

        out_file = os.path.join(out_dir, fname + '.png')
        if os.path.isfile(out_file):
            out_file = os.path.join(out_dir, fname + '_1.png')
        io.log(f'  Saving figure to: {out_file}')
        g.savefig(out_file, dpi=150, bbox_inches='tight')
        
        if not os.path.isfile(out_file):
            plt.savefig(out_file, dpi=150, bbox_inches='tight')
            
        print(f'Time spent plotting: {time.time() - t1:.3g}s.\n')
    print(80*'-' + '\n')
            
print(80 * '-' + '\n\n')
print(f'Time to complete: {time.time() - t0:.4g}s.')

In [ ]:
import time
from plotters.plot_utils import bootstrap
import shutil
import datetime
from plotters.plot_utils import load_pkl
from plotters.plot_observables import get_run_dirs, get_title_str
from lattice.lattice import u1_plaq_exact
import matplotlib.patches as patches
np.set_printoptions(edgeitems=3,infstr='inf',
                    linewidth=75, nanstr='nan', precision=8,
                    suppress=False, threshold=1000, formatter=None)

now = datetime.datetime.now()
day_str = now.strftime('%Y_%m_%d')
hour_str = now.strftime('%H%M')
time_str = f'{day_str}_{hour_str}'
from config import COLORS

colors = 100 * [*COLORS]

INFO_STR = None

t0 = time.time()
for idx, log_dir in enumerate(log_dirs):
    io.log(f'log_dir: {log_dir}')
    run_dirs = sorted(get_run_dirs(log_dir))[::-1]
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    lf = params['num_steps']
    clip_value = params.get('clip_value', 0)
    px = {}
    plaqs_diffs = {}
    charges = {}

    tunneling_rates = {}
    tunneling_events = {}
    for run_dir in run_dirs:
        t1 = time.time()
        run_params = load_pkl(os.path.join(run_dir, 'run_params.pkl'))
        net_weights = run_params['net_weights']
        eps = run_params['eps']
        io.log(f'  net_weights: {tuple(net_weights)}')
            
        px_ = load_pkl(os.path.join(run_dir, 'observables', 'px.pkl'))
        px_ = np.squeeze(np.array(px_))
        avg_px = np.mean(px_)
        if avg_px < 0.1 or tuple(net_weights) not in nw_include:
            print(f'  INFO: Skipping {tuple(net_weights)} since <px> = {avg_px}.')
            continue
        
        plaqs = load_pkl(os.path.join(run_dir, 'observables', 'plaqs.pkl'))
        dplq = u1_plaq_exact(run_params['beta']) - np.squeeze(np.array(plaqs))
        
        q = load_pkl(os.path.join(run_dir, 'observables', 'charges.pkl'))
        q = np.squeeze(np.array(q))
        
        px_ = px_[1:]
        dplq = dplq[1:]

        num_steps = px_.shape[0]
        therm_steps = int(0.2 * num_steps)
        
        q = q[therm_steps:]
        px_ = px_[therm_steps:]
        dplq = dplq[therm_steps:]

        dq, tr = calc_tunneling_rate(q)
        dq = dq.T

        n_boot = 1000
        px_avg, px_err, px_ = bootstrap(px_, n_boot=n_boot)
        dplq_avg, dplq_err, dplq_ = bootstrap(dplq, n_boot=n_boot)
        dq_avg, dq_err, dq_ = bootstrap(dq, n_boot=n_boot)
        
        #px_ = px_[:n_boot // 2]
        #dplq_ = dplq_[:n_boot // 2]
        #dq_ = dq_[:n_boot // 2]

        key = tuple(net_weights)
        px[key] = px_
        plaqs_diffs[key] = dplq_
        tunneling_rates[key] = tr
        tunneling_events[key] = dq_

        data = pd.DataFrame({
            'plaqs_diffs': plaqs_diffs[key].flatten(),
            'accept_prob': px[key].flatten(),
            'tunneling_rate': tunneling_events[key].flatten()
        })
        g = sns.PairGrid(data)
        cmap = sns.dark_palette(colors[idx], as_cmap=True)
        try:
            g = g.map_diag(sns.kdeplot, shade=True, color=colors[idx], gridsize=100)
            g = g.map_lower(sns.kdeplot, shade=False, cmap=cmap, Nchunk=5, gridsize=50)
        except:
            g = g.map_lower(plt.hist, histtype='step', color=colors[idx], alpha=0.6, ec=colors[idx], density=True)
            g = g.map_diag(plt.hist, histtype='step', color=colors[idx], alpha=0.6, ec=colors[idx], density=True)
        g = g.map_upper(plt.plot, color=colors[idx], ls='', marker=',', rasterized=True)#, alpha=1.)
        #g = g.map_offdiag(plt.plot, color=colors[idx], ls='', marker=',', alpha=0.75)
        title_str = (r"$N_{\mathrm{LF}} = $" + f'{lf}, '
                     r"$\varepsilon = $"  + f'{eps:.3g}')
        if params['clip_value'] > 0:
            title_str += f', clip: {clip_value}'
        title_str += f', nw: {key}'
        g.fig.suptitle(title_str, y=1.02, fontsize='x-large')
        
        out_dir = os.path.abspath(f'/home/foremans/cooley_figures/pairplots_{time_str}')
        if INFO_STR is not None:
            out_dir = os.path.join(out_dir, INFO_STR)
        io.check_else_make_dir(out_dir)
        
        nw_str = ''.join((str(i) for i in key))
        fname = f'lf{lf}_'
        if clip_value > 0:
            fname += f'clip{int(clip_value)}_'
        fname += f'_{nw_str}'

        out_file = os.path.join(out_dir, fname + '.png')
        if os.path.isfile(out_file):
            out_file = os.path.join(out_dir, fname + '_1.png')
        io.log(f'  Saving figure to: {out_file}')
        g.savefig(out_file, dpi=150, bbox_inches='tight')
        
        if not os.path.isfile(out_file):
            plt.savefig(out_file, dpi=150, bbox_inches='tight')
            
        print(f'Time spent plotting: {time.time() - t1:.3g}s.\n')
    print(80*'-' + '\n')
            
print(80 * '-' + '\n\n')
print(f'Time to complete: {time.time() - t0:.4g}s.')

## g.fig.axes[0].tick_params.reset=


In [ ]:
%debug

In [ ]:
from plotters.plot_utils import bootstrap
import shutil
import datetime
from plotters.plot_utils import load_pkl
from plotters.plot_observables import get_run_dirs, get_title_str
from lattice.lattice import u1_plaq_exact
import matplotlib.patches as patches
np.set_printoptions(edgeitems=3,infstr='inf',
                    linewidth=75, nanstr='nan', precision=8,
                    suppress=False, threshold=1000, formatter=None)

now = datetime.datetime.now()
day_str = now.strftime('%Y_%m_%d')
hour_str = now.strftime('%H%M')
time_str = f'{day_str}_{hour_str}'

for log_dir in log_dirs:
    io.log(f'log_dir: {log_dir}')
    run_dirs = get_run_dirs(log_dir)
    plaqs_diffs = get_obs_dict(log_dir, 'plaqs', run_dirs=run_dirs)
    px = get_obs_dict(log_dir, 'px', run_dirs=run_dirs)
    charges = get_obs_dict(log_dir, 'charges', run_dirs=run_dirs)
    px = {k: px[k] for k in nw_include}
    charges = {k: charges[k] for k in nw_include}
    plaqs_diffs = {k: plaqs_diffs[k] for k in nw_include}
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    lf = params['num_steps']
    clip_value = params.get('clip_value', 0)

    tunneling_rates = {}
    tunneling_events = {}
    for key in plaqs_diffs.keys():
        io.log(f'  net_weights: {key}')
        dplq = np.squeeze(np.array(plaqs_diffs[key]))
        px_ = np.squeeze(np.array(px[key]))
        q = np.squeeze(np.array(charges[key]))
        dq, tr = calc_tunneling_rate(q)
        
        dq = dq.T
        tr = tr[1:]
        px_ = px_[1:]
        dplq = dplq[1:]

        num_steps = px_.shape[0]
        therm_steps = int(0.2 * num_steps)
        dq = dq[therm_steps:]
        tr = tr[therm_steps:]
        px_ = px[therm_steps:]
        dplq = dplq[therm_steps:]
        
        px[key] = px_
        plaqs_diffs[key] = dplq
        tunneling_rates[key] = tr
        tunneling_events[key] = dq

        #px_avg, px_err, px_ = bootstrap(therm_arr(px[key]), n_boot=1000)
        #tr_avg, tr_err, tr_ = bootstrap(therm_arr(tunneling_rates[key]), n_boot=1000)
        #dq_avg, dq_err, dq_ = bootstrap(therm_arr(tunneling_events[key]), n_boot=1000)
        #plqd_avg, plqd_err, plqd_ = bootstrap(therm_arr(plaqs_diffs[key]), n_boot=1000)

        plqd_ = plqd_.flatten()
        px_ = px_.flatten()
        tr_ = tr_.flatten()
        dq_ = dq_.flatten()
        data = pd.DataFrame({'plaqs_diffs': plqd_, 'accept_prob': px_, 'tunneling_events': dq_})
        try:
            g = sns.PairGrid(data)
            g = g.map_diag(sns.kde, shade=True, color='C6')
            g = g.map_offdiag(plt.plot, color='C6', ls='', marker=',', alpha=0.75)
        except:
            g = sns.PairGrid(data)
            g = g.map_diag(plt.hist, histtype='stepfilled', color='C6')

        g = sns.JointGrid(x=plqd_, y=px_, ratio=3) #, space=0)
        g = g.plot_joint(plt.plot, color='C0', ls='', marker=',', alpha=0.75)
        try:
            g = g.plot_marginals(sns.kdeplot, color='C0', shade=True)
        except:
            g = g.plot_marginals(plt.hist, density=True,
                                 histtype='stepfilled',
                                 ec='C0', alpha=0.6)

        text_str = r"$N_{\mathrm{LF}} = $" + f'{lf}\n{key}'
        if params['clip_value'] > 0:
            text_str += f'\n clip: {clip_value}'


        bbox_props = dict(boxstyle="round", fc="w", ec="0.5", alpha=0.9)
        g.fig.text(x=0.19, y=0.15, s=text_str,
                   ha='left', va='bottom',
                   fontsize='large', bbox=bbox_props)
        g.set_axis_labels(xlabel=r"""$\delta \phi_{P}$""", ylabel=r"""$A(\xi|\xi^{\prime})$""")
        plt.tight_layout()
        

        #out_dir = os.path.abspath(f'home/foremans/cooley_figures/jointplots_{time_str}')
        out_dir = os.path.abspath(f'/home/foremans/cooley_figures/jointplots_{time_str}')
        nw_str = ''.join((str(i) for i in key))
        io.check_else_make_dir(out_dir)
        fname = f'plaqs_diffs_accept_rates_lf{lf}_{nw_str}'
        if clip_value > 0:
            fname += f'_clip{int(clip_value)}'

        out_file = os.path.join(out_dir, fname + '.png')
        if os.path.isfile(out_file):
            out_file = os.path.join(out_dir, fname + '_1.png')
        io.log(f'  Saving figure to: {out_file}')
        g.fig.savefig(fname + '.png', dpi=200, bbox_inches='tight')
        
        g.fig.close()
        plt.close('all')
        

        g = sns.JointGrid(x=plqd_, y=dq_, ratio=3) #, space=0)
        g = g.plot_joint(plt.plot, color='C6', ls='', marker=',', alpha=0.75)
        try:
            g = g.plot_marginals(sns.kdeplot, color='C6', shade=True)
        except:
            g = g.plot_marginals(sns.distplot, kde=False,
                                 density=True,
                                 histtype='stepfilled',
                                 ec='C6', alpha=0.6)
            plt.hist()

        g.fig.text(x=0.19, y=0.15, s=text_str,
                   ha='left', va='bottom',
                   fontsize='large', bbox=bbox_props)
        g.set_axis_labels(xlabel=r"""$\delta \phi_{P}$""", ylabel=r"""$\gamma$""")
        plt.tight_layout()

        fname = f'plaqs_diffs_tunneling_rates_lf{lf}_{nw_str}'
        if clip_value > 0:
            fname += f'_clip{int(clip_value)}'

        out_file = os.path.join(out_dir, fname + '.png')
        if os.path.isfile(out_file):
            out_file = os.path.join(out_dir, fname + '_1.png')
        io.log(f'  Saving figure to: {out_file}')
        g.fig.savefig(out_file, dpi=200, bbox_inches='tight')
        io.log('\n')

In [ ]:
pd.DataFrame.from_dict()


In [ ]:
d_ = {
    'plaqs_diffs': [*plaqs_diffs[(0, 0, 0, 0, 0, 0)].flatten()],
    'px': [*px[(0, 0, 0, 0, 0, 0)].flatten()],
    'dq': [*tunneling_events[(0, 0, 0, 0, 0, 0)].flatten()],
}
df = pd.DataFrame(d_)

In [ ]:
df

In [ ]:
sns.pairplot(df)

In [ ]:
g = sns.PairGrid(df)
g.map_diag(sns.kdeplot)
g.map_offdiag(sns.kdeplot, n_levels=6);

In [ ]:
%debug

In [ ]:
tunneling_events[(0, 0, 0, 0, 0, 0)].shape

In [ ]:
g.fig.axes

In [ ]:
#key = (1, 1, 1, 1, 1, 1)
for key in plaqs_diffs.keys():
    print(f'Making jointplot with {key}...')
    plqd_avg, plqd_err, plqd_ = bootstrap(therm_arr(plaqs_diffs[key]), n_boot=5000, ci=68)
    px_avg, px_err, px_ = bootstrap(therm_arr(px[key]), n_boot=5000)
    tr_avg, tr_err, tr_ = bootstrap(therm_arr(tunneling_rates[key]), n_boot=5000)
    dq_avg, dq_err, dq_ = bootstrap(therm_arr(tunneling_events[key]), n_boot=5000)

    plqd_ = plqd_.flatten()
    px_ = px_.flatten()
    tr_ = tr_.flatten()
    dq_ = dq_.flatten()
    #xlim = (plqd_avg - np.min(plqd_), plqd_avg + np.max(plqd_))
    #ylim = (px_avg - np.min(px_), px_avg + np.max(px_))

    g = sns.JointGrid(x=plqd_, y=dq_, ratio=3) #, space=0)
    g = g.plot_joint(plt.scatter, color='C0', alpha=0.75, edgecolor='white')
    #g = g.plot_marginals(sns.distplot, kde=True, color='C0')
    g = g.plot_marginals(sns.kdeplot, color='C0', shade=True)
    g.fig.text(x=0.13, y=0.13, s=f'nw: {key}', fontsize='large')
    #g.plot(plt.scatter, sns.kdeplot)
    #_ = g.ax_marg_x.hist(px_, color='C6', alpha=0.6,
    #                     orientation='horizontal', histtype='stepfilled')
    #_ = g.ax_marg_x.hist(plqd_, color='C6', alpha=0.6,
    #                     orientation='horizontal', histtype='stepfilled')
    g.set_axis_labels(xlabel=r"""$\delta \phi_{P}$""", ylabel=r"""$\gamma$""")
    plt.tight_layout()
    #g.fig.suptitle(f'net weights: {key}', y=1.02)

In [ ]:
g.fig.text()

In [ ]:
%debug

In [ ]:
for key in plaqs_diffs.keys():
    px_ = therm_arr(np.squeeze(px[key]))
    dplq_ = therm_arr(np.squeeze(plaqs_diffs[key]))
    px_ = np.mean(px_, axis=0)
    dplq_ = np.mean(dplq_, axis=0)
    
    g = sns.jointplot(x=px_,
                      y=dplq_)
    g = g.plot_joint(plt.scatter, color='gray', edgecolor='white')
    _ = g.ax_marg_x.hist(px_, color='C0', alpha=0.6,
                         orientation='horizontal', histtype='stepfilled')
    _ = g.ax_marg_x.hist(dplq_, color='C1', alpha=0.6,
                         orientation='horizontal', histtype='stepfilled')
    #g.annotate(key)
    
    #g = g.plot_marginals(sns.kdeplot, shade=True)

    g.set_axis_labels(xlabel=r"""$\delta \phi_{P}$""", ylabel=r"""$A(\xi|\xi^{\prime}$""")
    g.fig.suptitle(key, y=1.02)
    
    #g = sns.jointplot(x=np.array(plaqs_diffs[key]),
    #                 y=np.array(tunn_rates[key]))
    #g = g.plot_marginals(sns.kdeplot, shade=True)
    #g.set_axis_labels(xlabel=r"""$\delta \phi_{P}$""", ylabel=r"""$\gamma$""")
    #g.fig.suptitle(key, y=1.02)

In [ ]:
import datetime
from plotters.plot_utils import load_pkl
from plotters.plot_observables import get_run_dirs, get_title_str
from lattice.lattice import u1_plaq_exact
#plaqs_plot_dir = os.path.join(../g ,'plaqs_plots')
#px_plot_dir = os.path.join('.' ,'px_plots')
now = datetime.datetime.now()
day_str = now.strftime('%Y_%m_%d')
hour_str = now.strftime('%H%M')

#tunn_rates_plot_dir = os.path.abspath(
#    f'../../gauge_logs/tunn_rates_hists_{day_str}_{hour_str}'
#)
tunn_rates = []
plaqs_diffs = []
px = []

for log_dir in log_dirs:
    print(f'Loading from log_dir: \n  {log_dir}')
    run_dirs = get_run_dirs(log_dir, filter_str='steps5000')
    try:
        tunn_rates_ = get_obs_dict(log_dir, 'tunn_rates', run_dirs)
        tunn_rates_ = {k: np.squeeze(np.array(tunn_rates_[k])) for k in nw_include}

        plaqs = get_obs_dict(log_dir, 'plaqs', run_dirs)
        #plaqs_diffs_  = {
        #    k: np.squeeze(np.array(u1_plaq_exact(5.) - plaqs[k])) for k in nw_include
        #}
        px_ = get_obs_dict(log_dir, 'px', run_dirs)
        px_ = {k: np.squeeze(px_[k]) for k in nw_include}
        tunn_rates.append(tunn_rates)
        plaqs_diffs.append(plaqs_diffs)
        px.append(px_)
    except KeyError:
        continue
    #fig, axes = plt.subplots(4, 4)
    #axes = axes.flatten()
    #for idx, key in plaqs_diffs.keys():
    #    ax = axes[idx]
    #    plaqs_ = plaqs_diffs[key]
    #    px_ = px[key]
    #    ax
    


In [ ]:
def therm_arr(arr, therm_frac=0.2):
    step_axis = np.argmax(arr.shape)
    num_steps = arr.shape[step_axis]
    therm_steps = int(therm_frac * num_steps)
    arr = np.delete(arr, np.s_[:therm_steps], axis=step_axis)
    
    return arr

for key in plaqs_diffs.keys():
    px_ = therm_arr(np.squeeze(px[key]))
    dplq_ = therm_arr(np.squeeze(plaqs_diffs[key]))
    px_ = np.mean(px_, axis=0)
    dplq_ = np.mean(dplq_, axis=0)
    
    g = sns.jointplot(x=px_,
                      y=dplq_)
    g = g.plot_joint(plt.scatter, color='gray', edgecolor='white')
    _ = g.ax_marg_x.hist(px_, color='C0', alpha=0.6,
                         orientation='horizontal', histtype='stepfilled')
    _ = g.ax_marg_x.hist(dplq_, color='C1', alpha=0.6,
                         orientation='horizontal', histtype='stepfilled')
    #g.annotate(key)
    
    #g = g.plot_marginals(sns.kdeplot, shade=True)

    g.set_axis_labels(xlabel=r"""$\delta \phi_{P}$""", ylabel=r"""$A(\xi|\xi^{\prime}$""")
    g.fig.suptitle(key, y=1.02)
    
    #g = sns.jointplot(x=np.array(plaqs_diffs[key]),
    #                 y=np.array(tunn_rates[key]))
    #g = g.plot_marginals(sns.kdeplot, shade=True)
    #g.set_axis_labels(xlabel=r"""$\delta \phi_{P}$""", ylabel=r"""$\gamma$""")
    #g.fig.suptitle(key, y=1.02)

In [ ]:
log_dir = log_dirs[0]
px_ = np.array(px[0][(1, 1, 1, 1, 1, 1)])
px_.shape
dplq_ = np.array(plaqs_diffs[0][(1, 1, 1, 1, 1, 1)])
dplq_.shape
tr_ = np.array(tunn_rates[0][(1, 1, 1, 1, 1, 1)])

g = sns.jointplot(px_, dplq_)

In [ ]:
%debug

In [ ]:
import datetime
from plotters.plot_utils import load_pkl
from plotters.plot_observables import get_run_dirs, get_title_str
from lattice.lattice import u1_plaq_exact
#plaqs_plot_dir = os.path.join(../g ,'plaqs_plots')
#px_plot_dir = os.path.join('.' ,'px_plots')
now = datetime.datetime.now()
day_str = now.strftime('%Y_%m_%d')
hour_str = now.strftime('%H%M')

tunn_rates_plot_dir = os.path.abspath(
    f'../../gauge_logs/tunn_rates_hists_{day_str}_{hour_str}'
)

for log_dir in log_dirs:
    run_dirs = get_run_dirs(log_dir, filter_str='steps5000')
    tunn_rates = get_obs_dict(log_dir, 'tunn_rates', run_dirs)
    tunn_rates = {k: tunn_rates[k] for k in nw_include}

    fig, axes = grid_plot(log_dir=log_dir,
                          obs_dict=tunn_rates,
                          run_dirs=run_dirs,
                          out_dir=tunn_rates_plot_dir,
                          obs_name='tunneling rate',
                          filter_str='steps5000',
                          plot_type='hist')

In [ ]:
import pandas as pd
from plotters.plot_utils import load_pkl
from plotters.plot_observables import get_run_dirs, get_title_str
from lattice.lattice import u1_plaq_exact
import scipy
import datetime
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsOrig)
#sns.set()
sns.set_context('notebook')
mpl.rcParams.update(mpl.rcParamsOrig)
mpl.rcParams.update(mpl.rcParamsDefault)
sns.set_palette('bright')
sns.set_style('ticks')
sns.set(mpl.rcParamsOrig)
sns.set(mpl.rcParamsDefault)
#sns.set_style('whitegrid')

sem = scipy.stats.sem
tunneling_rates_ = {}
plaq_diffs_ = {}

now = datetime.datetime.now()
day_str = now.strftime('%Y_%m_%d')
hour_str = now.strftime('%H%M')

data = {}
for idx, log_dir in enumerate(log_dirs):
    params = load_pkl(os.path.join(log_dir, 'parameters.pkl'))
    run_dirs = sorted(get_run_dirs(log_dir, filter_str='steps5000'))

    plaqs_diffs_dict = {}
    dq_dict = {}
    tunn_rates_dict = {}
    run_dirs_dict = {}
    tr_arr = []
    tr_err = []
    dq_arr = []
    pld_arr = []
    pld_err = []
    #for idx, (k ,v) in enumerate(charges.items()):
    
    for run_dir in run_dirs:
        #run_dir = run_dirs[idx]
        px = np.squeeze(np.array(load_pkl(os.path.join(run_dir, 'observables', 'px.pkl'))))
        if np.mean(px) > 0.1:
            run_params = load_pkl(os.path.join(run_dir, 'run_params.pkl'))
            nw = tuple(run_params['net_weights'])
            plaqs = np.squeeze(np.array(load_pkl(os.path.join(run_dir, 'observables', 'plaqs.pkl'))))
            charges = np.squeeze(np.array(load_pkl(os.path.join(run_dir, 'observables', 'charges.pkl'))))
            num_steps = plaqs.shape[0]
            therm_steps = int(0.2 * num_steps)
            plaqs = plaqs[therm_steps:]
            charges = charges[therm_steps:]

            pld = plaqs - u1_plaq_exact(run_params['beta'])
            dq, tr = calc_tunneling_rate(charges)

            dq_dict[nw] = dq
            tunn_rates_dict[nw] = tr
            plaqs_diffs_dict[nw] = pld
            run_dirs_dict[nw] = run_dir

            tr_arr.append(np.mean(tr))
            tr_err.append(np.std(tr))
            dq_arr.append(np.mean(dq))
            pld_arr.append(np.mean(pld))
            pld_err.append(np.std(pld))

        '''
        out_file = os.path.join(run_dir, 'observables', 'plaqs_diffs.pkl')
        #print(f'Saving pld to: {out_file}.')
        with open(out_file, 'wb') as f:
            pickle.dump(pld, f)

        out_file = os.path.join(run_dir, 'observables', 'dq.pkl')
        #print(f'Saving dq to: {out_file}.')
        with open(out_file, 'wb') as f:
            pickle.dump(dq, f)

        out_file = os.path.join(run_dir, 'observables', 'tunn_rates.pkl')
        #print(f'Saving tr to: {out_file}.')
        with open(out_file, 'wb') as f:
            pickle.dump(tr, f)
        '''
            
    dq_arr = np.array(dq_arr)
    tr_arr = np.array(tr_arr)
    tr_err = np.array(tr_err)
    pld_arr = np.array(pld_arr)
    pld_err = np.array(pld_err)
    
    clip_value = params.get('clip_value', 0)
    lf = params['num_steps']
    data[(lf, clip_value)] = {
        'dq': dq_dict,
        'run_dirs': run_dirs_dict,
        'tunn_rates': tunn_rates_dict,
        'plaqs_diffs': plaqs_diffs_dict,
        'plaqs_diffs_stats': (pld_arr, pld_err),
        'tunn_rates_stats': (tr_arr, tr_err),
    }
    
    
    label = f"lf: {params['num_steps']}"
    clip_value = params.get('clip_value', 0)
    if clip_value > 0:
        label += f'clip: {int(clip_value)}'
    
    fig, ax = plt.subplots()
        
    _ = ax.errorbar(pld_arr, tr_arr,
                    xerr=pld_err/params['batch_size'],
                    yerr=tr_err/params['batch_size'],
                    label=label, ls='', alpha=0.5,
                    ecolor=colors[idx], color=colors[idx])
    _ = ax.scatter(pld_arr, tr_arr, marker='.', color=colors[idx], zorder=10)
    
    title_str = get_title_str(params, eps=run_params['eps'], 
                              beta=run_params['beta'], nw_legend=False)
    _ = ax.set_title(title_str) #, fontsize='x-large')
    #xlabel = (r"$\langle\phi_{\mathrm{P}}\rangle "
    #          r"- \phi_{\mathrm{P}}^{*}$")
    xlabel = r"""$\delta \phi_{P}$"""
    _ = ax.set_xlabel(xlabel) #, fontsize=14)
    _ = ax.set_ylabel(r"""$\gamma$""") #, fontsize=14)
    #_ = ax.figure.autofmt_xdate()
    plt.tight_layout()
    
    
    log_dir_str = log_dir.split('/')[-1]
    gld = os.path.abspath('../../gauge_logs/')
    lf_str = f"lf{params['num_steps']}"
    out_dir = os.path.join(gld, f'tunneling_rates_{day_str}_{hour_str}')
    #out_dir = os.path.join(', 'tunneling_rates_vs_plaq_diffs', log_dir_str)
    #out_dir = os.path.join(log_dir, 'figures')
    io.check_else_make_dir(out_dir)
    fname = f"tunn_rate_vs_bias_lf{params['num_steps']}"
    
    clip_value = params.get('clip_value', 0)
    if clip_value > 0:
        fname += f'_clip{int(clip_value)}'
        
    #_ = ax.legend(loc='best')
        
    out_file = os.path.join(out_dir, fname + '.png')
    #all_file = os.path.join(out_dir, fname + '_all.png')
    print(f'Saving figure to: {out_file}.')
    plt.savefig(out_file, dpi=200, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()

colors_dict = {
    1: 'C0',
    2: 'C1',
    3: 'C2',
    4: 'C3',
    5: 'C4'
}
for idx, (key, val) in enumerate(data.items()):
    lf, clip_value = key
    pld_avg, pld_err = val['plaqs_diffs_stats']
    tr_avg, tr_err = val['tunn_rates_stats']
    tr_avg = np.mean(tr_avg)
    tr_err = np.mean(tr_err)
    pld_avg = np.mean(pld_avg)
    pld_err = np.mean(pld_err)
    if clip_value > 0:
        label = r"""$N_{\mathrm{LF}}^{*} = $""" + f'{lf}'
        marker = 's'
        fillstyle='none'
    else:
        label = r"""$N_{\mathrm{LF}} = $""" + f'{lf}'
        marker = 'o'
        fillstyle='full'
        
    _ = ax.errorbar(pld_avg, tr_avg, xerr=pld_err, yerr=tr_err,
                    ls='', ecolor=colors_dict[lf], color=colors_dict[lf], alpha=0.5)
    _ = ax.plot(pld_avg, tr_avg, label=label, marker=marker,
                ls='', fillstyle=fillstyle, color=colors_dict[lf], zorder=10)
    
    #title_str = get_title_str(params, eps=run_params['eps'], beta=run_params['beta'], nw_legend=False)
    #_ = ax.set_title(title_str, fontsize='x-large')
    xlabel = (r"$\langle\phi_{\mathrm{P}}\rangle "
              r"- \phi_{\mathrm{P}}^{*}$")
    _ = ax.set_xlabel(xlabel, fontsize=14)
    _ = ax.set_ylabel('tunneling rate', fontsize=14)
    #ylim = ax.get_ylim()
    #_ = ax.set_ylim((ylim[0], 0.010))
    
    plt.tight_layout()
    
    
    
    #log_dir_str = log_dir.split('/')[-1]
    out_dir = os.path.join('.', 'tunneling_rates_vs_plaq_diffs')
    io.check_else_make_dir(out_dir)
    fname = f'tunneling_rate_vs_plaq_diff_all'
    
    _ = ax.legend(loc='best', ncol=2, fontsize='small')
        
out_file = os.path.join(out_dir, fname + '.png')
#all_file = os.path.join(out_dir, fname + '_all.png')
print(f'Saving figure to: {out_file}.')
plt.savefig(out_file, dpi=200, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()

colors_dict = {
    1: 'C0',
    2: 'C1',
    3: 'C2',
    4: 'C3',
    5: 'C4'
}
for idx, (key, val) in enumerate(data.items()):
    lf, clip_value = key
    pld_avg, pld_err = val['plaqs_diffs_stats']
    tr_avg, tr_err = val['tunn_rates_stats']
    tr_avg = np.mean(tr_avg)
    tr_err = np.mean(tr_err)
    pld_avg = np.mean(pld_avg)
    pld_err = np.mean(pld_err)
    if clip_value > 0:
        label = r"""$N_{\mathrm{LF}}^{*} = $""" + f'{lf}'
        marker = 's'
        fillstyle='none'
    else:
        label = r"""$N_{\mathrm{LF}} = $""" + f'{lf}'
        marker = 'o'
        fillstyle='full'
        
    _ = ax.errorbar(pld_avg, tr_avg, xerr=pld_err, yerr=tr_err,
                    ls='', ecolor=colors_dict[lf], color=colors_dict[lf], alpha=0.5)
    _ = ax.plot(pld_avg, tr_avg, label=label, marker=marker,
                ls='', fillstyle=fillstyle, color=colors_dict[lf], zorder=10)
    
    #title_str = get_title_str(params, eps=run_params['eps'], beta=run_params['beta'], nw_legend=False)
    #_ = ax.set_title(title_str, fontsize='x-large')
    xlabel = (r"$\langle\phi_{\mathrm{P}}\rangle "
              r"- \phi_{\mathrm{P}}^{*}$")
    _ = ax.set_xlabel(xlabel, fontsize=14)
    _ = ax.set_ylabel('tunneling rate', fontsize=14)
    ylim = ax.get_ylim()
    _ = ax.set_ylim((ylim[0], 0.010))
    
    plt.tight_layout()
    
    
    
    #log_dir_str = log_dir.split('/')[-1]
    out_dir = os.path.join('.', 'tunneling_rates_vs_plaq_diffs')
    io.check_else_make_dir(out_dir)
    fname = f'tunneling_rate_vs_plaq_diff_all'
    
    _ = ax.legend(loc='best', ncol=2, fontsize='small')
        
out_file = os.path.join(out_dir, fname + '_zoom.png')
#all_file = os.path.join(out_dir, fname + '_all.png')
print(f'Saving figure to: {out_file}.')
plt.savefig(out_file, dpi=200, bbox_inches='tight')

In [ ]:
figsize = (6.4 + 0.45, 4.8)

fig, ax = plt.subplots(figsize=figsize)

colors_dict = {
    1: 'C0',
    2: 'C1',
    3: 'C2',
    4: 'C3',
    5: 'C4'
}
for idx, (key, val) in enumerate(data.items()):
    lf, clip_value = key
    pld_avg, pld_err = val['plaqs_diffs_stats']
    tr_avg, tr_err = val['tunn_rates_stats']
    tr_avg = np.mean(tr_avg)
    tr_err = np.mean(tr_err)
    pld_avg = np.mean(pld_avg)
    pld_err = np.mean(pld_err)
    if clip_value > 0:
        label = r"""$N_{\mathrm{LF}}^{*} = $""" + f'{lf}'
        marker = 's'
        fillstyle='none'
    else:
        label = r"""$N_{\mathrm{LF}} = $""" + f'{lf}'
        marker = 'o'
        fillstyle='full'
        
    _ = ax.errorbar(pld_avg, tr_avg, xerr=pld_err, yerr=tr_err,
                    ls='', ecolor=colors_dict[lf], color=colors_dict[lf], alpha=0.5)
    _ = ax.plot(pld_avg, tr_avg, label=label, marker=marker,
                ls='', fillstyle=fillstyle, color=colors_dict[lf], zorder=10)
    
    #title_str = get_title_str(params, eps=run_params['eps'], beta=run_params['beta'], nw_legend=False)
    #_ = ax.set_title(title_str, fontsize='x-large')
    xlabel = (r"$\langle\phi_{\mathrm{P}}\rangle "
              r"- \phi_{\mathrm{P}}^{*}$")
    _ = ax.set_xlabel(xlabel, fontsize=14)
    _ = ax.set_ylabel('tunneling rate', fontsize=14)
    ylim = ax.get_ylim()
    _ = ax.set_ylim((ylim[0], 0.010))
    xlim = ax.get_xlim()
    _ = ax.set_xlim((-0.125, 0.04))
    plt.tight_layout()
    
    #log_dir_str = log_dir.split('/')[-1]
    out_dir = os.path.join('.', 'tunneling_rates_vs_plaq_diffs')
    io.check_else_make_dir(out_dir)
    fname = f'tunneling_rate_vs_plaq_diff_all'
    
    #_ = ax.legend(fontsize='small',  ncol=1, bbox_to_anchor=(1.4, 1))
    _ = ax.legend(loc='best', ncol=2, fontsize='small')
        
out_file = os.path.join(out_dir, fname + '_zoom1.png')
#all_file = os.path.join(out_dir, fname + '_all.png')
print(f'Saving figure to: {out_file}.')
plt.savefig(out_file, dpi=200, bbox_inches='tight')

In [ ]:
figsize = (6.4 + 0.45, 4.8)

fig, ax = plt.subplots(figsize=figsize, )
for idx, (key, val) in enumerate(data.items()):
    lf, clip_value = key
    pld_avg, pld_err = val['plaqs_diffs_stats']
    tr_avg, tr_err = val['tunn_rates_stats']
    tr_avg = np.mean(tr_avg)
    tr_err = np.mean(tr_err)
    pld_avg = np.mean(pld_avg)
    pld_err = np.mean(pld_err)
    label = r"""$N_{\mathrm{LF}} = $""" + f'{lf}'
    if clip_value > 0:
        label += f' clip: {clip_value}'
        
    _ = ax.errorbar(pld_avg, tr_avg, xerr=pld_err, yerr=tr_err, label=label,
                    ls='', ecolor=colors[idx], color=colors[idx], alpha=0.5)
    _ = ax.scatter(pld_avg, tr_avg, marker='.', color=colors[idx], zorder=10)
    
    #title_str = get_title_str(params, eps=run_params['eps'], beta=run_params['beta'], nw_legend=False)
    #_ = ax.set_title(title_str, fontsize='x-large')
    xlabel = (r"$\langle\phi_{\mathrm{P}}\rangle "
              r"- \phi_{\mathrm{P}}^{*}$")
    _ = ax.set_xlabel(xlabel, fontsize=14)
    _ = ax.set_ylabel('tunneling rate', fontsize=14)
    ylim = ax.get_ylim()
    _ = ax.set_ylim((ylim[0], 0.010))
    xlim = ax.get_xlim()
    _ = ax.set_xlim((-0.125, 0.04))
    
    plt.tight_layout()
    
    #log_dir_str = log_dir.split('/')[-1]
    out_dir = os.path.join('.', 'tunneling_rates_vs_plaq_diffs')
    io.check_else_make_dir(out_dir)
    fname = f'tunneling_rate_vs_plaq_diff_all'
    
    _ = ax.legend(fontsize='small',  ncol=1, bbox_to_anchor=(1.4, 1))
        
out_file = os.path.join(out_dir, fname + '_zoom1.png')
#all_file = os.path.join(out_dir, fname + '_all.png')
print(f'Saving figure to: {out_file}.')
plt.savefig(out_file, dpi=200, bbox_inches='tight')

In [ ]:
%matplotlib inline

In [ ]:
from plotters.plot_observables import grid_plot, get_obs_dict
import utils.file_io as io

out_dir = os.path.abspath('../../gauge_logs/figures_2019_12_17/tunneling_rates')
#out_dir = os.path.join('.', 'tunneling_rate_plots')
io.check_else_make_dir(out_dir)

for log_dir in log_dirs:
    charges = get_obs_dict(log_dir, 'charges')
    qdict = {k: np.array(v)[0] for k, v in charges.items()} 
    dq_dict = {}
    tr_dict = {}
    tr_tot_dict = {}

    for key, val in qdict.items():
        dq, tr = calc_tunneling_rate(val)
        dq_dict[key] = dq
        tr_dict[key] = tr
        tr_tot_dict[key] = dq.sum(axis=1)


    grid_plot(log_dir, obs_dict=tr_dict, therm_frac=0, 
              obs_name='tunneling_rate', axis=1, plot_type='hist', 
              filter_str='steps5000', stats=False, out_dir=out_dir)

    #grid_plot(log_dir, obs_dict=dq_dict, therm_frac=0., 
    #          obs_name='dq_all', axis=1, plot_type='hist', 
    #          filter_str='steps5000', stats=False, out_dir=out_dir)

    #grid_plot(log_dir, obs_dict=tr_tot_dict, therm_frac=0., 
    #          obs_name='tr_tot', axis=1, plot_type='hist', 
    #          filter_str='steps5000', stats=False, out_dir=out_dir)

In [ ]:
from plotters.plot_observables import grid_plot

log_dir = log_dirs[0]
charges = get_obs_dict(log_dir, 'charges')
qdict = {k: np.array(v)[0] for k, v in charges.items()} 
dq_dict = {}
tr_dict = {}
tr_tot_dict = {}

for key, val in qdict.items():
    dq, tr = calc_tunneling_rate(val)
    dq_dict[key] = dq
    tr_dict[key] = tr
    tr_tot_dict[key] = dq.sum(axis=1)


grid_plot(log_dir, obs_dict=tr_dict, therm_frac=0, 
          obs_name='tr', axis=1, plot_type='hist', 
          filter_str='steps5000', stats=False, out_dir='./testing')